# Proposal 2

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

2024-01-09 21:37:53.670916: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jesusoyanedel/.pyenv/versions/3.10.11/envs/tt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import CTSV1
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 100
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_2.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

100%|██████████| 9912422/9912422 [00:00<00:00, 150443130.19it/s]


Extracting /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw/train-images-idx3-ubyte.gz to /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 37549812.10it/s]

Extracting /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw/train-labels-idx1-ubyte.gz to /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 65873788.15it/s]

Extracting /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw/t10k-images-idx3-ubyte.gz to /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11469312.92it/s]

Extracting /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/jesusoyanedel/Documents/USM/tt2/datasets/TensorMNIST/raw



## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_l1': 0.1,
    'lambda_l2': 0.01,
    'eps': 1e-05
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),

    loggers=[InteractiveLogger()],
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = CTSV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:10<00:00, 42.98it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:11<00:00, 41.05it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:11<00:00, 40.51it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:11<00:00, 40.05it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:12<00:00, 37.29it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.40it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9119
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.36it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1053
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.75it/s]
> Eval on

  2%|▏         | 1/50 [02:34<2:06:26, 154.82s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1127
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1277
-- >> Start of training phase << --
100%|██████████| 469/469 [00:13<00:00, 35.96it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:13<00:00, 35.10it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:13<00:00, 35.90it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:13<00:00, 35.37it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:13<00:00, 35.96it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 40.56it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0182
	ExperienceForgetting/eval_phase/test_stream/

  4%|▍         | 2/50 [05:18<2:07:51, 159.81s/it]

...


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.0921
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6370
	StreamForgetting/eval_phase/test_stream = 0.6370
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3096
-- >> Start of training phase << --
100%|██████████| 469/469 [00:13<00:00, 34.76it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:12<00:00, 36.68it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:12<00:00, 37.07it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:12<00:00, 36.91it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:12<00:00, 36.14it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.62it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.7482
	ExperienceForgetting/eval_phase/test_stream

 98%|█████████▊| 49/50 [2:07:05<02:30, 150.90s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1396
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6464
	StreamForgetting/eval_phase/test_stream = 0.6464
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3056
-- >> Start of training phase << --
100%|██████████| 469/469 [00:12<00:00, 38.21it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:12<00:00, 38.33it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:12<00:00, 38.27it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:12<00:00, 38.61it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:12<00:00, 38.86it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 45.26it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.7800
	ExperienceForgetting/eval_phase/test_stream

100%|██████████| 50/50 [2:09:35<00:00, 155.51s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.9445
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6552
	StreamForgetting/eval_phase/test_stream = 0.6552
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3002


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [13]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))